# Predicting Match Outcomes

In [26]:
#SQL Imports
import mysql.connector
#Pandas imports
import pandas as pd
import datetime

In [27]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)
mycursor = mydb.cursor()

In [28]:
query = pd.read_sql_query('''
    SELECT * 
    FROM TeamMatchStats
''', mydb)

In [29]:
match_stat_df = pd.DataFrame(query)

In [31]:
match_stat_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,conversions,penalty_goals,conversion_percentage,...,prop_1,prop_2,hooker,sr_1,sr_2,lock_1,int_1,int_2,int_3,int_4
0,1,1,14,1,1,34,5,4,3,0.8000,...,404,396,391,407,406,483,1626,402,397,219
1,2,1,1,14,0,12,2,1,1,0.5000,...,11,500,499,2,10,269,3,400,9,1624
2,3,2,8,6,1,19,3,3,0,1.0000,...,192,503,201,196,188,197,190,429,194,505
3,4,2,6,8,0,18,3,2,1,0.6667,...,510,509,153,511,155,512,516,309,513,327
4,5,3,10,4,1,20,4,1,1,0.2500,...,268,272,265,264,375,266,260,276,267,261


# Prediction Algorithm

Things to try:
1. Want to create scoring function (sklearn.metrics import f1_score, make_scorer, classification_report)

2. Random Forest
3. Neural Network (10-3-2)

Features to try:
- home field advantage
- form in last X matches
- Points differential last X matches
- Points scored last X matches
- Points allowed last X matches
- Running metre differential last X matches
- Tackle Percentages
- Factor in the strength of opposition


In [33]:
correlation_df = match_stat_df
col = list(match_stat_df.columns)[35:52]
correlation_df = match_stat_df.drop(col, axis=1)
correlation_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,conversions,penalty_goals,conversion_percentage,...,prop_1,prop_2,hooker,sr_1,sr_2,lock_1,int_1,int_2,int_3,int_4
0,1,1,14,1,1,34,5,4,3,0.8000,...,404,396,391,407,406,483,1626,402,397,219
1,2,1,1,14,0,12,2,1,1,0.5000,...,11,500,499,2,10,269,3,400,9,1624
2,3,2,8,6,1,19,3,3,0,1.0000,...,192,503,201,196,188,197,190,429,194,505
3,4,2,6,8,0,18,3,2,1,0.6667,...,510,509,153,511,155,512,516,309,513,327
4,5,3,10,4,1,20,4,1,1,0.2500,...,268,272,265,264,375,266,260,276,267,261


In [34]:
correlation_df.corr()

,id,match_id,team_id,opponent_id,is_winner,points,tries,conversions,penalty_goals,field_goals,...,prop_1,prop_2,hooker,sr_1,sr_2,lock_1,int_1,int_2,int_3,int_4
id,1.000000,0.999994,-0.002829,-0.002881,-0.000529,0.010396,0.037230,0.068866,-0.163577,-0.019496,...,0.093227,0.008069,-0.128755,0.011474,-0.108848,0.082351,-0.098295,0.037413,-0.080819,-0.087647
match_id,0.999994,1.000000,-0.002855,-0.002855,0.000000,0.010887,0.037673,0.069293,-0.163378,-0.019474,...,0.093370,0.007961,-0.128786,0.011372,-0.108939,0.082331,-0.098228,0.037302,-0.080524,-0.087751
team_id,-0.002829,-0.002855,1.000000,-0.049816,0.092365,-0.009626,-0.000808,-0.026427,-0.006392,-0.042795,...,0.305901,-0.046469,0.458025,0.254933,0.090401,0.374976,0.331167,0.129452,0.122383,0.130072
opponent_id,-0.002881,-0.002855,-0.049816,1.000000,-0.092365,-0.084622,-0.086244,-0.071843,-0.021973,-0.048408,...,0.054150,-0.046887,-0.034943,-0.029696,-0.031758,-0.028862,0.013022,-0.042624,0.010509,-0.092111
is_winner,-0.000529,0.000000,0.092365,-0.092365,1.000000,0.616395,0.563085,0.495853,0.331492,0.213446,...,0.007433,-0.030531,0.011110,0.070447,0.091329,0.023509,0.074850,-0.017668,-0.020505,-0.031842
points,0.010396,0.010887,-0.009626,-0.084622,0.616395,1.000000,0.964721,0.918395,0.238678,0.024725,...,0.084750,0.015578,0.005018,0.006640,0.034028,0.007198,0.071831,-0.061336,-0.026747,0.019000
tries,0.037230,0.037673,-0.000808,-0.086244,0.563085,0.964721,1.000000,0.862692,0.038343,-0.032242,...,0.070016,0.022999,0.017176,0.010454,0.039848,0.009256,0.069899,-0.056634,-0.032004,0.032896
conversions,0.068866,0.069293,-0.026427,-0.071843,0.495853,0.918395,0.862692,1.000000,0.060914,-0.017794,...,0.096460,-0.023847,0.001098,-0.007565,0.030453,0.024880,0.040990,-0.048905,-0.021766,0.014144
penalty_goals,-0.163577,-0.163378,-0.006392,-0.021973,0.331492,0.238678,0.038343,0.060914,1.000000,0.125693,...,0.038834,0.025896,-0.045731,0.005154,-0.013962,-0.027066,0.049898,-0.058646,0.019954,-0.028058
field_goals,-0.019496,-0.019474,-0.042795,-0.048408,0.213446,0.024725,-0.032242,-0.017794,0.125693,1.000000,...,-0.076203,0.050405,-0.015431,-0.042294,-0.008354,-0.040236,0.012491,0.049286,0.016749,-0.038017


In [ ]:
for match in list(match_stat_df['match_id'].unique()):
    match_df = match_stat_df[match_stat_df['match_id'] == match]
    #print(match_df)
    for team in list(match_df['team_id']):
        team_data = match_df[match_df['team_id'] == team]
        opp_data = match_df[match_df['team_id'] != team]
#         opp_points = opp_data['points'][1]
#         point_differential = team_data['points'][0] - opp_points
#         opp_tries = opp_data['tries']
#         opp_runs = opp_data['runs']
#         opp_run_metres = opp_data['run_metres']
#         opp_metres_per_run = opp_data['metres_per_run']
#         opp_kick_return_metres = opp_data['kick_return_metres']
#         opp_post_contact_metres = opp_data['post_contact_metres']
#         opp_line_breaks = opp_data['line_breaks']
#         opp_tackles = opp_data['tackles']
#         opp_missed_tackles = opp_data['missed_tackles'] + opp_data['ineffective_tackles']
#         opp_kicks = opp_data['kicks']
#         opp_kicking_metres = opp_data['metres_per_kick']
        #print(opp_data['points'].dtype)
        #points = team_data['points']
        #print(points)
        #opp_points = opp_data['points']
        #point_diff = points - opp_points
        point_diff = team_data['points'].iloc[0] - opp_data['points'].iloc[0]
        run_metre_diff = team_data['run_metres'].iloc[0] - opp_data['run_metres'].iloc[0]
        opp_missed_tackles = opp_data['missed_tackles'].iloc[0] + opp_data['ineffective_tackles'].iloc[0]
        metres_per_run_diff = float(team_data['metres_per_run'].iloc[0]) - float(opp_data['metres_per_run'].iloc[0])
        
        print(metres_per_run_diff)

1.3541000000000007
-1.3541000000000007
-1.8365
1.8365
-0.3675999999999995
0.3675999999999995
0.5433000000000003
-0.5433000000000003
9.999999999976694e-05
-9.999999999976694e-05
-1.080400000000001
1.080400000000001
1.5291999999999994
-1.5291999999999994
-0.5739000000000001
0.5739000000000001
-0.6962999999999999
0.6962999999999999
3.321300000000001
-3.321300000000001
2.5736
-2.5736
-0.03480000000000061
0.03480000000000061
-0.2812999999999999
0.2812999999999999
0.35770000000000035
-0.35770000000000035
0.8363000000000005
-0.8363000000000005
-1.0067000000000004
1.0067000000000004
-0.8142000000000005
0.8142000000000005
-0.9245000000000001
0.9245000000000001
-0.46169999999999867
0.46169999999999867
-0.28309999999999924
0.28309999999999924
1.2424999999999997
-1.2424999999999997
-0.9049999999999994
0.9049999999999994
-0.7059999999999995
0.7059999999999995
1.6693000000000016
-1.6693000000000016
-0.7157999999999998
0.7157999999999998
-0.5607000000000006
0.5607000000000006
-0.43240000000000123
0.4

In [ ]:
mycursor.close()